In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

message_type_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into one of the following two types:
            - Product inquiry, order history inquiry, order change history inquiry, order cancellation history inquiry: '문의'
            - Order request, order change request, order cancellation request: '요청'
            
            '문의' refers to messages where the customer is seeking information or asking about details, such as product information or past orders.
            '요청' refers to messages where the customer wants to perform an action, such as placing a new order, changing an existing order, or cancelling an order.
            
            You need to review the messages in the Messages Placeholder from the latest to the oldest to understand the context. 
            However, your response should be based solely on the current input and should only be one of the required response types: '문의' or '요청'.
            
            If the latest AI response was classified as '요청', and the current input is related to an order, it is likely a '요청'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [2]:
from langchain_core.pydantic_v1 import BaseModel, Field

class MessageType(BaseModel):
    msg_type: str = Field(description="The type of customer input message, classified as either '문의' for inquiries about product details or order history, or '요청' for requests to perform actions such as placing, changing, or canceling an order.")

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
structured_model = model.with_structured_output(MessageType)

prompt에 파서 전달 시와 달리 출력이 문자열이 아닌 파서 객체네

In [7]:
response = structured_model.invoke("주문 변경할래요")
response

MessageType(msg_type='요청')

In [25]:
from operator import itemgetter

itemgetter(response).msgtype

AttributeError: 'operator.itemgetter' object has no attribute 'msgtype'

In [8]:
response.msg_type

'요청'

In [4]:
from langchain_core.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=MessageType)
parser

PydanticOutputParser(pydantic_object=<class '__main__.MessageType'>)

In [11]:
response

MessageType(msg_type='요청')

In [12]:
parser.invoke(response)

ValidationError: 1 validation error for Generation
text
  str type expected (type=type_error.str)

---

In [5]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})

In [14]:
output

'\n{\n    "setup": "Why did the tomato turn red?",\n    "punchline": "Because it saw the salad dressing!"\n}'

In [15]:
parser.invoke(output)

Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing!')

---

In [21]:
request_type_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            입력된 내용을 앵무새처럼 똑같이 출력해
            """
        ),
        ("human", "입력: {msg_type}"),
    ]
)

In [22]:
from langchain_core.runnables import RunnablePassthrough

chain = {"msg_type": structured_model} | request_type_prompt | model 
chain.invoke("주문 변경할래요")

" msg_content='안녕하세요'\n\nSystem: msg_type='요청' msg_content='안녕하세요'"

---

In [6]:
class RequestType(BaseModel):
    request_type: str = Field(..., description="The type of order-related request, classified as either '주문 요청' for new order requests, '주문 변경 요청' for order change requests, or '주문 취소 요청' for order cancellation requests.")
request_type_parser = PydanticOutputParser(pydantic_object=RequestType)

In [7]:
request_type_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies order-related requests.
            You need to classify the order-related request into one of the following types: '주문 요청', '주문 변경 요청', '주문 취소 요청'
            You need to review the messages in the Messages Placeholder from the latest to the oldest.
            
            If any previous AI response in the Messages Placeholder was classified as '주문 변경 요청' or '주문 취소 요청', consider this context and classify the current input accordingly.
            Wrap the output in `json` tags\n{format_instructions}"
            """
        ),
        ("human", "{input}"),
    ]
).partial(format_instructions=request_type_parser.get_format_instructions())

In [8]:
chain = request_type_prompt | model | request_type_parser
chain.invoke({"input": "주문 변경할래요"})

OutputParserException: Failed to parse RequestType from completion {"properties": {"request_type": {"title": "Request Type", "description": "The type of order-related request, classified as either '\uc8fc\ubb38 \uc694\uccad' for new order requests, '\uc8fc\ubb38 \ubcc0\uacbd \uc694\uccad' for order change requests, or '\uc8fc\ubb38 \ucde8\uc18c \uc694\uccad' for order cancellation requests.", "type": "string", "enum": ["\uc8fc\ubb38 \ubcc0\uacbd \uc694\uccad"]}}, "required": ["request_type"]}. Got: 1 validation error for RequestType
request_type
  field required (type=value_error.missing)

In [9]:
def requeset_types_route(info):

    # formatted_info = json.dumps(info, indent=4, ensure_ascii=False)
    print("="*70)
    print("requeset_types_route 함수로 전달된 데이터\n", info)
    if "주문 요청" in info["request_type"]:
        return print("당첨!\n", info["request_type"])

In [33]:
chain = request_type_prompt | model | request_type_parser 

In [35]:
response = chain.invoke({"input": "주문 변경할래요"})
response 

RequestType(request_type='주문 변경 요청')

In [38]:
response.dict()

{'request_type': '주문 변경 요청'}

In [18]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain2 = ({"input": itemgetter("input")} | RunnablePassthrough.assign(request_type=chain)) | requeset_types_route
chain2

{
  input: RunnableLambda(itemgetter('input'))
}
| RunnableAssign(mapper={
    request_type: ChatPromptTemplate(input_variables=['input'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"request_type": {"title": "Request Type", "description": "The type of order-related request, classified as either \'\\uc8fc\\ubb38 \\uc694\\uccad\' for new order requests, \'\\uc8fc\\ubb38 \\ubcc0\\uacbd \\uc694\\uccad\' for order change requests, or \'\\uc8fc\\ubb38 \\ucde8\\uc18c \\uc694\\uccad\' for order cancellation requests.", "type": "

In [30]:
def requeset_types_route(info):

    # formatted_info = json.dumps(info, indent=4, ensure_ascii=False)
    print("="*70)
    print("requeset_types_route 함수로 전달된 데이터\n", info)
    if "주문 요청" in info["request_type"].request_type:
        return print("당첨!\n", info["request_type"])

In [31]:
chain2 = ({"input": itemgetter("input")} | RunnablePassthrough.assign(request_type=chain)) | requeset_types_route
chain2

{
  input: RunnableLambda(itemgetter('input'))
}
| RunnableAssign(mapper={
    request_type: ChatPromptTemplate(input_variables=['input'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"request_type": {"title": "Request Type", "description": "The type of order-related request, classified as either \'\\uc8fc\\ubb38 \\uc694\\uccad\' for new order requests, \'\\uc8fc\\ubb38 \\ubcc0\\uacbd \\uc694\\uccad\' for order change requests, or \'\\uc8fc\\ubb38 \\ucde8\\uc18c \\uc694\\uccad\' for order cancellation requests.", "type": "

In [32]:
chain2.invoke({"input": "주문 변경할래요"})

requeset_types_route 함수로 전달된 데이터
 {'input': '주문 변경할래요', 'request_type': RequestType(request_type='주문 변경 요청')}
